In [1]:
from Data_Handler.DataReader import DataReader
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from impressions import Impressions
from hybrid import *
from tqdm import tqdm
from evaluator import evaluate
import pandas as pd
import numpy as np
# Read & split data
dataReader = DataReader()

In [2]:
target = dataReader.load_target()

URM = dataReader.load_augmented_binary_urm()
URM_aug,icm = dataReader.pad_with_zeros_ICMandURM(URM)

URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample(URM_aug, train_percentage = 0.9)
URM_train_pow = dataReader.stackMatrixes(URM_train_aug)

>>> number of target users: 41116
4877


In [3]:
UserKNNCF = UserKNNCFRecommender(URM_train_aug)
UserKNNCF.fit()

RP3beta_pow = RP3betaRecommender(URM_train_pow)
RP3beta_pow.fit(alpha=0.3648761546066018,beta=0.5058870363874656, topK=480, normalize_similarity=True)

S_SLIM = SLIMElasticNetRecommender(URM_train_pow)
S_SLIM.fit()

EASE_R = EASE_R_Recommender(URM_train_pow)
EASE_R.fit()

UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 41629 (100.0%), 655.53 column/sec. Elapsed time 1.06 min
RP3betaRecommender: Similarity column 27968 (100.0%), 1640.48 column/sec. Elapsed time 17.05 sec
SLIMElasticNetRecommender: Processed 7153 (25.6%) in 5.00 min. Items per second: 23.84


KeyboardInterrupt: 

In [ ]:
# Instantiate and fit hybrid recommender
recommender = HybridRecommender_5(URM_train_aug, URM_train_pow,RP3beta_pow, S_SLIM, EASE_R, UserKNNCF)
#recommender = HybridRecommender(URM_train)
recommender.fit(UserKNNCF_tier1_weight=0.5729283925580592, RP3beta_pow_tier1_weight= 0.5126995921400189, EASE_R_tier1_weight=0.35135124299835835, UserKNNCF_tier2_weight=0.9573738859277221, RP3beta_pow_tier2_weight=0.8130000344620287, EASE_R_tier2_weight=0.8407574723860113, RP3beta_pow_tier3_weight=0.8153126010122613, S_SLIM_tier3_weight=0.39864407118121825, EASE_R_tier3_weight=0.8181461648227059, S_SLIM_tier4_weight= 0.3513706238839294, EASE_R_tier4_weight= 0.009916836090967942)

Hybrid_Recommender_4: URM Detected 3461 (12.4%) items with no interactions.
UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 41629 (100.0%), 746.35 column/sec. Elapsed time 55.78 sec
RP3betaRecommender: Similarity column 27968 (100.0%), 1735.38 column/sec. Elapsed time 16.12 sec
SLIMElasticNetRecommender: Processed 7799 (27.9%) in 5.00 min. Items per second: 25.99
SLIMElasticNetRecommender: Processed 15854 (56.7%) in 10.00 min. Items per second: 26.42
SLIMElasticNetRecommender: Processed 23623 (84.5%) in 15.00 min. Items per second: 26.24
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 15.89 min. Items per second: 29.33


In [ ]:
# Create CSV for submission
f = open("submission.csv", "w+")
f.write("user_id,item_list\n")
recommended_items_for_each_user = {}
for user_id in tqdm(target):
    recommended_items = recommender.recommend(user_id, cutoff=10, remove_seen_flag=True)
    #recommended_items=impressions.update_ranking(user_id,recommended_items,dataReader)
    recommended_items_for_each_user[int(user_id)]=recommended_items
    well_formatted = " ".join([str(x) for x in recommended_items])
    f.write(f"{user_id}, {well_formatted}\n")

100%|██████████| 41116/41116 [00:49<00:00, 832.46it/s]


In [ ]:
# Evaluare recommended items
map=evaluate(recommended_items_for_each_user,URM_validation,target)
print('MAP score: {}'.format(map))

100%|██████████| 41116/41116 [00:00<00:00, 51844.75it/s]

MAP score: 0.040385383312785086
